In [1]:
%matplotlib inline

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import torchvision
import torchvision.utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
import os

from dataset import next_batch

In [9]:
class PyTorchSiameseNetwork(nn.Module):
  
    def __init__(self, output_size):

        super(PyTorchSiameseNetwork, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )

        self.model = nn.Sequential(
            conv_bn(  3,  32, 2), 
            conv_dw( 32,  64, 1),
            conv_dw( 64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7, padding=1),
        )
        self.output_fully_connected = nn.Linear(1024, output_size)

    def forward_once(self, input_batch):
        net = self.model(input_batch)
        net = net.view(-1, 1024)
        return self.output_fully_connected(net)

    def forward(self, input_1, input_2):
        output_1 = self.forward_once(input_1)
        output_2 = self.forward_once(input_2)
        return output_1, output_2

In [3]:
class ContrastiveLoss(nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [4]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)


def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)


def imshow(img, text=None, size=8):
  
    np_img = img.numpy()
    width, height = np_img.shape

    fig = plt.figure(figsize=(size * height / width, size))

    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic', fontweight='bold',
                 bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np_img)
    plt.show()        

In [10]:
class SiameseModel(object):

    def __init__(self,
                 input_image_shape,
                 output_size=2,
                 margin=0.2,
                 learning_rate=0.01):
        
        self.target_shape = input_image_shape
        
        # Load the model and nn modules.
        if torch.cuda.is_available():
            self.network = PyTorchSiameseNetwork(output_size)
            self.network = self.network.cuda()
            
            self.criterion = ContrastiveLoss(margin=margin).cuda()
            self.optimiser = optim.SGD(self.network.parameters(), 
                                       lr=learning_rate,
                                       nesterov=True,
                                       momentum=0.99)
        else:
            raise ValueError('You need a GPU to for this network!')

    def inference(self, x):
        """Here the net computes an embedding for the input image(s)."""
        
        # Convert to variables on the GPU.
        width, height, channels = self.target_shape
        pytorch_shape = (-1, channels, width, height)
        
        # NumPy to PyTorch.
        x = to_var(torch.from_numpy(x.reshape(pytorch_shape)))
        return self.network.forward_once(x)

    def optimise_batch(self, batch_left, batch_right, batch_similar):
        
        # Initialise the gradients buffers.
        self.optimiser.zero_grad()

        # Convert to variables on the GPU.
        width, height, channels = self.target_shape
        pytorch_shape = (-1, channels, height, width)
        
        # Reshape for pytorch model.
        batch_left = batch_left.reshape(pytorch_shape)
        batch_right = batch_right.reshape(pytorch_shape)
        
        # NumPy to PyTorch.
        batch_left = to_var(torch.from_numpy(batch_left))
        batch_right = to_var(torch.from_numpy(batch_right))
        batch_similar = to_var(torch.from_numpy(batch_similar))  
        
        # Get encoding vectors.
        encoding_left, encoding_right = self.network(batch_left, batch_right)
        
        # Loss calculated and backpropagate error.
        loss = self.criterion(encoding_left, encoding_right, batch_similar)
        loss.backward()
        self.optimiser.step()
        
        return loss.data[0]

    def save(self, path):
        pass

In [12]:
dataset_folder = './sign_dataset/'

embedding_size = 512
learning_rate = 0.00001

training_iterations = 5000
save_step = 100
batch_size = 4

image_height = 256
image_width = 256
image_channels = 3
image_shape = (image_height, image_width, image_channels)

batch_generator = next_batch(batch_size,
                             data_directory=dataset_folder,
                             target_shape=image_shape,
                             probability=0.1)

model = SiameseModel(image_shape,
                     output_size=embedding_size,
                     margin=0.2,
                     learning_rate=learning_rate)

for batch_left, batch_right, batch_similar in batch_generator:

    loss = model.optimise_batch(batch_left, batch_right, batch_similar)
    print("loss: {}".format(loss))

Initialised with 296 image(s) found.
Output directory set to ./sign_dataset/kfh/output.Initialised with 342 image(s) found.
Output directory set to ./sign_dataset/barnard_marcus/output.Initialised with 1014 image(s) found.
Output directory set to ./sign_dataset/ocean/output.Initialised with 297 image(s) found.
Output directory set to ./sign_dataset/conran_estates/output.Initialised with 370 image(s) found.
Output directory set to ./sign_dataset/acorn/output.Initialised with 678 image(s) found.
Output directory set to ./sign_dataset/peter_james/output.Initialised with 512 image(s) found.
Output directory set to ./sign_dataset/winkworth/output.Initialised with 611 image(s) found.
Output directory set to ./sign_dataset/openrent/output.Initialised with 841 image(s) found.
Output directory set to ./sign_dataset/goodman_lilley/output.Initialised with 408 image(s) found.
Output directory set to ./sign_dataset/house_simple/output.Initialised with 435 image(s) found.
Output directory set to ./s

loss: 0.15021148324012756
loss: 0.16761162877082825
loss: 0.15439873933792114
loss: 0.17680957913398743
loss: 0.16331475973129272
loss: 0.14748841524124146
loss: 0.16318175196647644
loss: 0.19280001521110535
loss: 0.1733713299036026
loss: 0.18977969884872437
loss: 0.16463565826416016
loss: 0.1482296884059906
loss: 0.1477217972278595
loss: 0.15732520818710327
loss: 0.17250794172286987
loss: 0.16187795996665955
loss: 0.16932108998298645
loss: 0.15115231275558472
loss: 0.16970381140708923
loss: 0.15296736359596252
loss: 0.16201180219650269
loss: 0.14722990989685059
loss: 0.15454623103141785
loss: 0.16832315921783447
loss: 0.1609274297952652
loss: 0.17066624760627747
loss: 0.1435217559337616
loss: 0.1656894087791443
loss: 0.15535596013069153
loss: 0.15420806407928467
loss: 0.15750019252300262
loss: 0.1567486673593521
loss: 0.17746087908744812
loss: 0.14664971828460693
loss: 0.1503259837627411
loss: 0.16621680557727814
loss: 0.1537371724843979
loss: 0.16179972887039185
loss: 0.1598317027091

loss: 0.1312847137451172
loss: 0.1394033432006836
loss: 0.1356787383556366
loss: 0.13986685872077942
loss: 0.12995651364326477
loss: 0.14788967370986938
loss: 0.1340654194355011
loss: 0.12695829570293427
loss: 0.12989193201065063
loss: 0.1477949023246765
loss: 0.133465975522995
loss: 0.13581889867782593
loss: 0.11688597500324249
loss: 0.12185385823249817
loss: 0.13616426289081573
loss: 0.12758371233940125
loss: 0.1435907483100891
loss: 0.14267465472221375
loss: 0.11421537399291992
loss: 0.1325773447751999
loss: 0.14907753467559814
loss: 0.12693247199058533
loss: 0.14268024265766144
loss: 0.13710059225559235
loss: 0.14146621525287628
loss: 0.13517603278160095
loss: 0.1378166377544403
loss: 0.1551859974861145
loss: 0.14027898013591766
loss: 0.15727508068084717
loss: 0.1337645947933197
loss: 0.12514640390872955
loss: 0.15229099988937378
loss: 0.1324257254600525
loss: 0.13039693236351013
loss: 0.13403983414173126
loss: 0.1322312355041504
loss: 0.13449010252952576
loss: 0.13343259692192078


loss: 0.11650292575359344
loss: 0.14106568694114685
loss: 0.1184048056602478
loss: 0.14030809700489044
loss: 0.11285237967967987
loss: 0.12446705996990204
loss: 0.1201842725276947
loss: 0.11342369765043259
loss: 0.15886543691158295
loss: 0.12295393645763397
loss: 0.10670486092567444
loss: 0.12918028235435486
loss: 0.13090208172798157
loss: 0.12352180480957031
loss: 0.14134752750396729
loss: 0.12409886717796326
loss: 0.1264207661151886
loss: 0.13098418712615967
loss: 0.13257229328155518
loss: 0.11576521396636963
loss: 0.12415280938148499
loss: 0.11856865882873535
loss: 0.12012845277786255
loss: 0.12257277220487595
loss: 0.12605562806129456
loss: 0.11933231353759766
loss: 0.11604148149490356
loss: 0.11227288842201233
loss: 0.11953575909137726
loss: 0.13477282226085663
loss: 0.12278157472610474
loss: 0.13061992824077606
loss: 0.13074330985546112
loss: 0.11234493553638458
loss: 0.1276744306087494
loss: 0.10856667160987854
loss: 0.12409354001283646
loss: 0.12270881980657578
loss: 0.11292394

loss: 0.10798086225986481
loss: 0.12069027125835419
loss: 0.10947403311729431
loss: 0.1253204643726349
loss: 0.11642025411128998
loss: 0.11076071858406067
loss: 0.11925336718559265
loss: 0.10593929141759872
loss: 0.10501357913017273
loss: 0.10154479742050171
loss: 0.10615280270576477
loss: 0.11012674868106842
loss: 0.11486244201660156
loss: 0.1121281236410141
loss: 0.10964725911617279
loss: 0.11617674678564072
loss: 0.12241321802139282
loss: 0.11607910692691803
loss: 0.10963989049196243
loss: 0.11969166994094849
loss: 0.1294860541820526
loss: 0.13126680254936218
loss: 0.12348619103431702
loss: 0.11386072635650635
loss: 0.10847984999418259
loss: 0.12178598344326019
loss: 0.11086530238389969
loss: 0.10723040997982025
loss: 0.11498714983463287
loss: 0.1046532616019249
loss: 0.11696118116378784
loss: 0.11340248584747314
loss: 0.10920824110507965
loss: 0.10684368014335632
loss: 0.10753067582845688
loss: 0.11771038174629211
loss: 0.11024507880210876
loss: 0.10994291305541992
loss: 0.11105958

...

loss: 0.01724589616060257
loss: 0.020593512803316116
loss: 0.017655450850725174
loss: 0.017857126891613007
loss: 0.019573822617530823
loss: 0.02044592797756195
loss: 0.01850246638059616
loss: 0.018267417326569557
loss: 0.0198432095348835
loss: 0.020337410271167755
loss: 0.017434224486351013
loss: 0.017579499632120132
loss: 0.019833102822303772
loss: 0.019121943041682243
loss: 0.017101909965276718
loss: 0.01801426336169243
loss: 0.017370514571666718
loss: 0.01989142596721649
loss: 0.02041858248412609
loss: 0.017739657312631607
loss: 0.019349075853824615
loss: 0.018276453018188477
loss: 0.021691836416721344
loss: 0.01711878553032875
loss: 0.018609855324029922
loss: 0.014738041907548904
loss: 0.01590464450418949
loss: 0.01966381072998047
loss: 0.020535584539175034
loss: 0.018456269055604935
loss: 0.018494855612516403
loss: 0.021378062665462494
loss: 0.01970970444381237
loss: 0.01583726890385151
loss: 0.018574733287096024
loss: 0.01785186305642128
loss: 0.019424093887209892
loss: 0.0171339

loss: 0.01660662516951561
loss: 0.015231424942612648
loss: 0.016366101801395416
loss: 0.016157781705260277
loss: 0.016000254079699516
loss: 0.01364703569561243
loss: 0.015008023008704185
loss: 0.015526272356510162
loss: 0.01694042980670929
loss: 0.016106905415654182
loss: 0.0164012648165226
loss: 0.01650191843509674
loss: 0.016069889068603516
loss: 0.01689637452363968
loss: 0.017537016421556473
loss: 0.015132895670831203
loss: 0.01716073602437973
loss: 0.014861566945910454
loss: 0.018065698444843292
loss: 0.014837806113064289
loss: 0.01494313683360815
loss: 0.016526710242033005
loss: 0.016789909452199936
loss: 0.015616808086633682
loss: 0.019431425258517265
loss: 0.015615282580256462
loss: 0.016684673726558685
loss: 0.015001147985458374
loss: 0.015301378443837166
loss: 0.01678675226867199
loss: 0.013052421621978283
loss: 0.01566758006811142
loss: 0.01403422188013792
loss: 0.016981493681669235
loss: 0.01683906465768814
loss: 0.016908351331949234
loss: 0.014194460585713387
loss: 0.014276

loss: 0.013140424154698849
loss: 0.015185371041297913
loss: 0.014278711751103401
loss: 0.0144589152187109
loss: 0.015094103291630745
loss: 0.012836350128054619
loss: 0.012033870443701744
loss: 0.013629499822854996
loss: 0.014507591724395752
loss: 0.014350424520671368
loss: 0.015937652438879013
loss: 0.013598929159343243
loss: 0.014361286535859108
loss: 0.017022212967276573
loss: 0.012168984860181808
loss: 0.01565304771065712
loss: 0.01760258898139
loss: 0.01250595785677433
loss: 0.01541686151176691
loss: 0.01545084360986948
loss: 0.016884297132492065
loss: 0.014463135041296482
loss: 0.014221381396055222
loss: 0.012986268848180771
loss: 0.013526389375329018
loss: 0.012559765949845314
loss: 0.015353454276919365
loss: 0.0146689647808671
loss: 0.015503935515880585
loss: 0.013862399384379387
loss: 0.013630353845655918
loss: 0.015997443348169327
loss: 0.013560816645622253
loss: 0.01331373956054449
loss: 0.0146463792771101
loss: 0.012815089896321297
loss: 0.01568455994129181
loss: 0.015128005

loss: 0.012007627636194229
loss: 0.013110007159411907
loss: 0.013292251154780388
loss: 0.01177988387644291
loss: 0.011866331100463867
loss: 0.013555007055401802
loss: 0.013019456528127193
loss: 0.012503702193498611
loss: 0.014695901423692703
loss: 0.012107129208743572
loss: 0.012633342295885086
loss: 0.012804754078388214
loss: 0.012173744849860668
loss: 0.011348666623234749
loss: 0.013614101335406303
loss: 0.011385012418031693
loss: 0.011833930388092995
loss: 0.01248839870095253
loss: 0.014907204546034336
loss: 0.014136075973510742
loss: 0.012271381914615631
loss: 0.01442775409668684
loss: 0.012533734552562237
loss: 0.013707530684769154
loss: 0.01264355331659317
loss: 0.013917174190282822
loss: 0.013867165893316269
loss: 0.01285269670188427
loss: 0.012829795479774475
loss: 0.014500169083476067
loss: 0.014423871412873268
loss: 0.01319429837167263
loss: 0.013517213985323906
loss: 0.015266954898834229
loss: 0.01340256817638874
loss: 0.012028960511088371
loss: 0.012642203830182552
loss: 0.

KeyboardInterrupt: 